In [ ]:
import numpy as np
import os
import pickle as pk
from sklearn.decomposition import PCA
import xarray as xr

# set the path to the training data
path='~/ForceSMIP/'

In [ ]:
def list_files(model, variable, domain):
    return os.listdir(path+'Training-Ext/'+domain+'/'+variable+'/'+model)

def fit_pca(model,variable,domain,variable_name):
    files=list_files(model, variable, domain)
    i=0
    time_lag=5*12
    features_raw=[]
    ds=xr.open_dataset(path+'/'+domain+'/'+variable+'/'+model+'/'+files[i])
    for j in range(0,len(ds.time)-time_lag,10):
        feature=ds.isel(time=slice(j,j+time_lag))[variable_name].values.flatten()
        feature=np.nan_to_num(feature, nan=0)
        features_raw.append(feature)
    pca = PCA(n_components=10)
    pca.fit(features_raw)
    pk.dump(pca, open('PCA_data/'+model+'_'+variable+'_'+'pca.pkl','wb'))
    features=[]
    times=[]
    for i in range(1,len(files)):
        print(i/len(files))
        ds=xr.open_dataset(path+'/'+domain+'/'+variable+'/'+model+'/'+files[i])
        for j in range(len(ds.time)-time_lag):
            times.append(j+time_lag)
            feature=ds.isel(time=slice(j,j+time_lag))[variable_name].values.flatten()
            feature=np.nan_to_num(feature, nan=0).reshape(1, -1)
            features.append(pca.transform(feature))
    np.save(path+'PCA_data/'+model+'_'+variable+'_'+'times.npy',times)
    np.save(path+'PCA_data/'+model+'_'+variable+'_'+'features.npy',features)

In [ ]:
variables=['pr','psl','tas','tos','monmaxtasmax','monmintasmin','monmaxpr']
variable_names=['pr','psl','tas','tos','tasmax','tasmin','pr']
domains=['Amon','Amon','Amon','Omon','Aday','Aday','Aday']
models=['CanESM5', 'CESM2', 'MIROC-ES2L', 'MIROC6', 'MPI-ESM1-2-LR']

for model in models:
    for i in range(len(variables)):
        fit_pca(model,variables[i],domains[i],variable_names[i])